<a href="https://colab.research.google.com/github/cxbxmxcx/PAIGCP/blob/master/PAIGCP_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time


In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [ ]:
import urllib.request
import re

url = "https://www.gutenberg.org/files/11/11-0.txt"
file = urllib.request.urlopen(url)
text = [line.decode('utf-8') for line in file]
text = ''.join(text)
text = re.sub(' +',' ',text)
text = re.sub(r'[^A-Za-z.,!\r ]+', '', text)
text = text[1150:]
text

'CHAPTER I.\rDown the RabbitHole\r\r\rAlice was beginning to get very tired of sitting by her sister on the\rbank, and of having nothing to do once or twice she had peeped into\rthe book her sister was reading, but it had no pictures or\rconversations in it, and what is the use of a book, thought Alice\rwithout pictures or conversations\r\rSo she was considering in her own mind as well as she could, for the\rhot day made her feel very sleepy and stupid, whether the pleasure of\rmaking a daisychain would be worth the trouble of getting up and\rpicking the daisies, when suddenly a White Rabbit with pink eyes ran\rclose by her.\r\rThere was nothing so very remarkable in that nor did Alice think it\rso very much out of the way to hear the Rabbit say to itself, Oh\rdear! Oh dear! I shall be late! when she thought it over afterwards,\rit occurred to her that she ought to have wondered at this, but at the\rtime it all seemed quite natural but when the Rabbit actually took a\rwatch out of its 

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
",".join(vocab)

'\r, ,!,,,.,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z'

In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
[f"{char} = {i}" for char,i in zip(char2idx, range(20))]

['\r = 0',
 '  = 1',
 '! = 2',
 ', = 3',
 '. = 4',
 'A = 5',
 'B = 6',
 'C = 7',
 'D = 8',
 'E = 9',
 'F = 10',
 'G = 11',
 'H = 12',
 'I = 13',
 'J = 14',
 'K = 15',
 'L = 16',
 'M = 17',
 'N = 18',
 'O = 19']

In [ ]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

[idx2char[i.numpy()] for i in char_dataset.take(5)]

['C', 'H', 'A', 'P', 'T']

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
[repr(''.join(idx2char[item.numpy()])) for item in sequences.take(5)]

["'CHAPTER I.\\rDown the RabbitHole\\r\\r\\rAlice was beginning to get very tired of sitting by her sister on th'",
 "'e\\rbank, and of having nothing to do once or twice she had peeped into\\rthe book her sister was reading'",
 "', but it had no pictures or\\rconversations in it, and what is the use of a book, thought Alice\\rwithout'",
 "' pictures or conversations\\r\\rSo she was considering in her own mind as well as she could, for the\\rhot '",
 "'day made her feel very sleepy and stupid, whether the pleasure of\\rmaking a daisychain would be worth '"]

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'CHAPTER I.\rDown the RabbitHole\r\r\rAlice was beginning to get very tired of sitting by her sister on t'
Target data: 'HAPTER I.\rDown the RabbitHole\r\r\rAlice was beginning to get very tired of sitting by her sister on th'


In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 7 ('C')
  expected output: 12 ('H')
Step    1
  input: 12 ('H')
  expected output: 5 ('A')
Step    2
  input: 5 ('A')
  expected output: 20 ('P')
Step    3
  input: 20 ('P')
  expected output: 24 ('T')
Step    4
  input: 24 ('T')
  expected output: 9 ('E')


In [ ]:
# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024
rnn_units_2 = 512


In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim,
                            batch_input_shape=[BATCH_SIZE, None]),
  tf.keras.layers.GRU(rnn_units,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'), 
  tf.keras.layers.GRU(rnn_units_2,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'),  
  tf.keras.layers.Dense(vocab_size)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           14592     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 512)           2362368   
_________________________________________________________________
dense (Dense)                (64, None, 57)            29241     
Total params: 6,344,505
Trainable params: 6,344,505
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 57) # (batch_size, sequence_length, vocab_size)


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 57)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.043874


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
24/24 [==============================] - 5s 198ms/step - loss: 3.3487
Epoch 2/50
24/24 [==============================] - 5s 197ms/step - loss: 2.7888
Epoch 3/50
24/24 [==============================] - 5s 197ms/step - loss: 2.4297
Epoch 4/50
24/24 [==============================] - 5s 198ms/step - loss: 2.2546
Epoch 5/50
24/24 [==============================] - 5s 195ms/step - loss: 2.1102
Epoch 6/50
24/24 [==============================] - 5s 199ms/step - loss: 1.9799
Epoch 7/50
24/24 [==============================] - 5s 198ms/step - loss: 1.8687
Epoch 8/50
24/24 [==============================] - 5s 199ms/step - loss: 1.7614
Epoch 9/50
24/24 [==============================] - 5s 197ms/step - loss: 1.6651
Epoch 10/50
24/24 [==============================] - 5s 198ms/step - loss: 1.5738
Epoch 11/50
24/24 [==============================] - 5s 195ms/step - loss: 1.4861
Epoch 12/50
24/24 [==============================] - 5s 196ms/step - loss: 1.4071
Epoch 13/50
24/24 [======

In [ ]:
def generate_text(model, start_string, temp, gen_chars):     
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)  
  text_generated = []

  # Here batch size == 1
  model.reset_states()
  for i in range(gen_chars):
      predictions = model(input_eval)      
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temp
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [ ]:
def generate_text(model, start_string, temp, gen_chars):     
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)  
  text_generated = []

  # Here batch size == 1
  model.reset_states()
  for i in range(gen_chars):
    predictions = model(input_eval)      
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temp
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])
  
  return (start_string + ''.join(text_generated))

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim,
                            batch_input_shape=[1, None]),
  tf.keras.layers.GRU(rnn_units,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'),
  tf.keras.layers.GRU(rnn_units_2,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'), 
  tf.keras.layers.Dense(vocab_size)
])
model.summary()
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            14592     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_3 (GRU)                  (1, None, 512)            2362368   
_________________________________________________________________
dense_1 (Dense)              (1, None, 57)             29241     
Total params: 6,344,505
Trainable params: 6,344,505
Non-trainable params: 0
_________________________________________________________________


In [ ]:
generate_text(model, u"Alice said ", 1.0, 200)

'Alice said nothing seem, I can e terms of this\ragreement and hold soup to tull him. A nice\rmuddle in the distance would take the place of the Mock Turtle is.\r\rIts the thing Mock Turtle Soup is macaid the King, t'